In [1]:
# 重启
#!kill -9 -1
# 挂载谷歌硬盘
from google.colab import drive
drive.mount('/content/drive')
# 创建数据文件夹
!mkdir /content/dataset
# 复制文件到本地
!cp /content/drive/My\ Drive/Dataset/netflix-prize-data.zip /content/dataset/netflix-prize-data.zip
# 切换当前目录
%cd /content/dataset/
# 解压缩文件
!unzip netflix-prize-data.zip
# 安装包
!pip install surprise

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
mkdir: cannot create directory ‘/content/dataset’: File exists
/content/dataset
Archive:  netflix-prize-data.zip
replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace combined_data_1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace combined_data_2.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace combined_data_3.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace combined_data_4.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace movie_titles.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace probe.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace qualifying.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [0]:
# 导入需要用到的包
import pandas as pd
import numpy as np

from surprise import Dataset
from surprise import Reader
from surprise import BaselineOnly
from surprise import accuracy
from surprise.model_selection import KFold

In [0]:
# 回收内存 
import gc
gc.collect()
# 查看内存中的变量
dir()
# %cd /content/dataset/

## 数据预处理

In [0]:
# 处理训练集的函数
def data_cleaning(data):
  data['movie_id']=data[0].map(lambda x:int(x[:-1]) if ':' in x else np.NaN)
  data=data.fillna(method='ffill')
  data=data[data[0].map(lambda x: ':' not in x)]
  data['user_id']=data[0].map(lambda x:x.split(',')[0]).astype(np.int)
  data['rating']=data[0].map(lambda x:x.split(',')[1]).astype(np.float)
  data=data.drop([data.columns[0]],axis=1)
  #data=data.set_index('user_id')

  return data
# 处理probe数据集的函数
def probe_cleaning(data):
  data['movie_id']=data[0].map(lambda x:int(x[:-1]) if ':' in x else np.NaN)
  data=data.fillna(method='ffill')
  data=data[data[0].map(lambda x: ':' not in x)]
  data.columns=['user_id','movie_id']
  data['user_id']=data['user_id'].astype(np.int)
  data=data[['movie_id','user_id']]

  return data

In [0]:
# 提取所有训练
data1=pd.read_csv('./combined_data_1.txt',sep=' ',header=None)
data2=pd.read_csv('./combined_data_2.txt',sep=' ',header=None)
data3=pd.read_csv('./combined_data_3.txt',sep=' ',header=None)
data4=pd.read_csv('./combined_data_4.txt',sep=' ',header=None)
data1 = data_cleaning(data1)
data2 = data_cleaning(data2)
data3 = data_cleaning(data3)
data4 = data_cleaning(data4)

In [7]:
# 将训练数据合并
data_all=pd.concat([data1,data2,data3,data4],ignore_index=True)
del data1, data2, data3, data4
data_all

,movie_id,user_id,rating
0,1.0,1488844,3.0
1,1.0,822109,5.0
2,1.0,885013,4.0
3,1.0,30878,4.0
4,1.0,823519,3.0
...,...,...,...
100480502,17770.0,1790158,4.0
100480503,17770.0,1608708,3.0
100480504,17770.0,234275,1.0
100480505,17770.0,255278,4.0


In [8]:
# 提取probe数据 作为测试集
probe=probe_cleaning(pd.read_csv('./probe.txt',sep=' ',header=None))
# probe数据集补全得分
probe=pd.merge(probe,data_all)
probe

,movie_id,user_id,rating
0,1.0,30878,4.0
1,1.0,2647871,4.0
2,1.0,1283744,3.0
3,1.0,2488120,5.0
4,1.0,317050,5.0
...,...,...,...
1408390,9997.0,2328701,3.0
1408391,9998.0,1288730,3.0
1408392,9998.0,2536567,4.0
1408393,9998.0,1107317,5.0


In [9]:
# 将probe数据从训练集中删除作为测试集
data_all=pd.concat([data_all,probe],ignore_index=True)
data_all=data_all.drop_duplicates(subset=['user_id','movie_id'],keep=False)
data_all

,movie_id,user_id,rating
0,1.0,1488844,3.0
1,1.0,822109,5.0
2,1.0,885013,4.0
4,1.0,823519,3.0
5,1.0,893988,3.0
...,...,...,...
100480502,17770.0,1790158,4.0
100480503,17770.0,1608708,3.0
100480504,17770.0,234275,1.0
100480505,17770.0,255278,4.0


In [12]:
# 压缩数据 m=4 和 n=4 可以压缩很多数据
def data_sample(data,m,n):
  data=data[data['user_id']%n==0]
  data=data[data['movie_id']%m==0]
  return data
data_all = data_sample(data_all,4,4)
probe = data_sample(probe,4,4)
data_all
probe

,movie_id,user_id,rating
26,1000.0,977808,3.0
31,1000.0,1960212,2.0
32,1000.0,97460,5.0
39,1000.0,929584,5.0
43,1000.0,2553920,3.0
...,...,...,...
1408378,9996.0,16792,3.0
1408379,9996.0,481320,4.0
1408381,9996.0,570792,4.0
1408383,9996.0,2571420,1.0


In [13]:
data_all

,movie_id,user_id,rating
2705,4.0,1544320,1.0
2707,4.0,732556,3.0
2710,4.0,303948,2.0
2711,4.0,2418760,4.0
2714,4.0,1519860,3.0
...,...,...,...
100472819,17768.0,558252,2.0
100472820,17768.0,780984,2.0
100472829,17768.0,295196,2.0
100472831,17768.0,1883788,3.0


## 训练模型

In [0]:
reader = Reader(line_format='item user rating', sep=',', skip_lines=1)
# load_from_df  直接从DataFrame 调用数据
data = Dataset.load_from_df(data_all[['movie_id', 'user_id', 'rating']], reader=reader)
del data_all
train_set = data.build_full_trainset()
# ALS优化
bsl_options = {'method': 'als','n_epochs': 5,'reg_u': 12,'reg_i': 5}
algo = BaselineOnly(bsl_options=bsl_options)

In [17]:
# Kfold交叉验证
kf = KFold(n_splits=3)
for trainset, testset in kf.split(data):
    # 训练并预测
    algo.fit(trainset)
    predictions = algo.test(testset)
    # 计算RMSE
    accuracy.rmse(predictions, verbose=True)

Estimating biases using als...
RMSE: 0.9260
Estimating biases using als...
RMSE: 0.9265
Estimating biases using als...
RMSE: 0.9258


In [19]:
# 在porbe数据集预测并评分
probe_test=[]

for i in probe.values:
    tup=(i[0],i[1],i[2])
    probe_test.append(tup)
# 在测试集预测
probe_predictions = algo.test(probe_test)
# 计算RMSE
accuracy.rmse(probe_predictions, verbose=True)   

RMSE: 0.9857


0.9857327938088634